In [ ]:
# CIFAR-10 Image Classification 

# Dataset of 50,000 32x32 color training images, labeled over 10 categories, and 10,000 test images.
# Ref : https://keras.io/datasets/

# https://www.cs.toronto.edu/~kriz/cifar.html


In [1]:
# import the CIFAR-10 datasets from Keras datasets repository
from tensorflow.keras.datasets import cifar10

In [ ]:
# load the data into environment 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# x_train, x_test: uint8 array of RGB image data with shape (num_samples, 3, 32, 32).
# y_train, y_test: uint8 array of category labels (integers in range 0-9) with shape (num_

 99672064/170498071 [================>.............] - ETA: 1:57

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# plot hte test image
plt.imshow(x_train[5000])
#plt.colorbar();

In [ ]:
# import models
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Convolution2D, MaxPooling2D , Dense, Dropout, Flatten  
import numpy as np


In [ ]:
batch_size = 1 # in each iteration, we consider 32 training examples at once
num_epochs = 20 # we iterate 20 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
conv_depth_1 = 6 # we will initially have 32 kernels per conv. layer...
pool_size = 2 # we will use 2x2 pooling throughout
#conv_depth_2 = 32 # ...switching to 64 after the first pooling layer
#drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
#drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 10 # the FC layer will have 512 neurons

In [ ]:
#### just renaming the x_input as per the code 
X_train = x_train 
X_test = x_test

X_train.shape


In [ ]:
y_train.shape

In [ ]:
X_train.shape

In [ ]:
num_train, height, width, depth = X_train.shape # there are 50000 training examples in CIFAR-10 
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

In [ ]:
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_test) # Normalise data to [0, 1] range

# Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
# Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

#Y_train 

In [ ]:
# from tensorflow.keras.utils import to_categorical
# Y_train = to_categorical(y_train, num_classes) # One-hot encode the labels
# Y_test =  to_categorical(y_test, num_classes) # One-hot encode the labels

In [ ]:
Y_train = y_train
Y_test  = y_test

In [ ]:
inp = Input(shape=(height, width, depth)) # depth goes last in TensorFlow back-end (first in Theano) - (32, 32, 3)

In [ ]:
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), activation='relu')(inp)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_1)

In [ ]:
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(pool_1)
hidden = Dense(hidden_size, activation='relu')(flat)
out = Dense(num_classes, activation='softmax')(hidden)

In [ ]:
model = Model(inputs=inp, outputs=out) 

In [29]:
#model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
model.compile(loss='sparse_categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 6)         168       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 6)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1350)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                13510     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
Total params: 13,788
Trainable params: 13,788
Non-trainable params: 0
_________________________________________________________

In [30]:
num_epochs=2

model.fit(X_train, Y_train,                # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

Train on 45000 samples, validate on 5000 samples
Epoch 1/2
45000/45000 [==============================] - 149s 3ms/sample - loss: 1.8963 - acc: 0.2919 - val_loss: 1.7034 - val_acc: 0.3926
Epoch 2/2
45000/45000 [==============================] - 146s 3ms/sample - loss: 1.5659 - acc: 0.4282 - val_loss: 1.5621 - val_acc: 0.4354


In [ ]:
model.evaluate(X_test, Y_test, verbose=1)  # Evaluate the trained model on the test set!